# Descobrindo o Setor Censitário de um DataFrame a partir da Malha Censitária do IBGE

### Antes de tudo deve-se fazer 2 coisas antes:
#### * Baixar a malha censitária mais atual https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/26565-malhas-de-setores-censitarios-divisoes-intramunicipais.html?=&t=acesso-ao-produto
#### * Lembrar que o codigo só irá identificar o setor se as coordenadas começarem com latitude e depois longitude

### 1. Transformar a coluna latitude e longitude do dataframe em Pontos(coordenadas) 

In [28]:
#Importar as bibliotecas
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [11]:
#Transformar um arquivo em um df (dataframe)
df = pd.read_excel('BR_Setores_2020/teste guilherme.xlsx')#Y = longitude e X = latitude


In [34]:
#Criar a coluna lat/long ou ponto ou coordenadas
df['lat/long'] = None

#Preencher a nova coluna com a latitude e longitude e depois as transformando em pontos (coordenadas)
for index, row in df.iterrows():
    df.loc[index, 'lat/long'] = Point(row.X, row.Y)
    
#Observando o resultado
df

C:\Users\Horus-User\Documents\Programas\Anaconda\lib\site-packages\pandas\core\internals\managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


### 2. Preenchendo a nova coluna de Setor Censitário do df com as informações do setor do IBGE

In [14]:
df['SC'] = None

In [13]:
#Planilha IBGE setor censitario
gdf_br = gpd.read_file('BR_Setores_2020/BR_Setores_2020.shp')


#### Nosso objetivo é que cada ponto passe por cada linha do 'CD_SETOR' e após isso, apagar a anterior e ir para a proxima. E caso o ponto ache o poligono em que está inserido, ele ira salvar o Setor Censitário ('CD_SETOR') desse poligono na nova coluna 'SC' do df (dataframe)

In [ ]:
for g in gdf_br.index:# g = linha
    for p in df.index:# p = linha
        if df['lat/long'][p].within(gdf_br['geometry'][g]):# Se coluna/linha = ponto esta dentro da coluna/linha = poligono
            df['SC'][p] = gdf_br['CD_SETOR'][g] # Escreva o setor censitario da linha em que o ponto esta dentro do poligono 
        #else:
            #gdf_br.drop(gdf_br.index[[g]], axis=0) # Apagar as linhas já analisadas para que o for nao passe por tudo toda vez

print(df['SC'])


C:\Users\Horus-User\AppData\Local\Temp\ipykernel_37060\1175502737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SC'][p] = gdf_br['CD_SETOR'][g] # Escreva o setor censitario da linha em que o ponto esta dentro do poligono


In [ ]:
#Checando o df
df

#### Vamos exportar para um arquivo excel e para um csv

In [ ]:
df.to_excel('teste_guilherme.xlsx', sheet_name = 'setores', index=False)
df.to_csv('teste_guilherme.csv', index=False)